In [25]:
# all required imports
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
from geopy.distance import geodesic
import json
import os
import sys
import math
import random
import pickle
import datetime
import scipy as sp
from scipy.stats.stats import pearsonr
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

In [5]:
print(os.listdir("./data/"))

['aminer_papers_11.txt', 'aminer_papers_1.txt', 'aminer_papers_0.txt', 'aminer_papers_10.txt', 'venues.csv', 'aminer_papers_12.txt', 'aminer_papers_2.txt', 'aminer_papers_3.txt', 'aminer_papers_13.txt', 'authors_df_sorted_by_num_citations_all.csv', 'aminer_papers_17.txt', 'aminer_papers_7.txt', 'aminer_papers_6.txt', 'aminer_papers_16.txt', 'aminer_papers_14.txt', 'aminer_papers_28.txt', 'aminer_papers_4.txt', 'aminer_papers_5.txt', 'aminer_papers_29.txt', 'aminer_papers_15.txt', 'authors.csv', 'dblp-ref', 'AMiner-Author.txt', 'authors_df_sorted_by_num_citations.csv', 'aminer_papers_18.txt', 'aminer_papers_24.txt', 'aminer_papers_8.txt', 'aminer_papers_9.txt', 'aminer_papers_25.txt', 'aminer_papers_19.txt', 'linking_relations.txt.zip', 'authors_sorted_by_h_index.csv', 'aminer_papers_27.txt', 'aminer_papers_26.txt', 'aminer_papers_22.txt', 'aminer_papers_23.txt', 'authors_sorted_by_h_index_all.csv', 'aminer_papers_21.txt', 'aminer_papers_20.txt']


In [22]:

keyword_dict = dict()
def read_aminer_author(filename):
    idx=0
    with open(filename) as f:
        lines = f.readlines()
        name = ""
        affiliation = ""
        keywords = ""
        for line in lines:
            line = line.rstrip()
            if line.startswith("#n "):
                line = line.split("#n ")[1]
                names = line.split(' ')
                name = names[0] + "_" + names[len(names)-1]
            elif line.startswith("#a "):
                affiliation = line.split("#a ")[1]
            elif line.startswith("#t "):
                keywords = line.split("#t ")[1]
            elif line.rstrip() == '':
                keyword_dict[name] = affiliation+":;:"+keywords
                
read_aminer_author("./data/AMiner-Author.txt")

In [39]:
authors_pickle = {}
with open('./data/auth_by_name.pkl', 'rb') as handle:
    authors_pickle = pd.read_pickle(handle)
authors_df = pd.DataFrame.from_dict(authors_pickle, orient='index', columns=['author_id', 'citations', 'co_authors', 'h_index', 'num_citations', 'paper_count'])



,author_id,citations,co_authors,h_index,num_citations,paper_count
"""lk""_G""rler",856763,50,[856762],1,25.000000,1
(Alex)_Meng,1311401,10,[41658],1,5.000000,1
(TYPE=name)_Ce,418610,610,"[69771, 133914, 8682, 418609, 1075, 35662, 125...",14,367.480952,20
-Ing._Marx-Gómez,259134,1,"[259132, 259133]",1,0.333333,1
-Jr._Drews,750197,19,"[496349, 170194, 97076, 185568]",1,3.800000,1
-Mali$#353_-Mali$#353,1460681,50,"[1460677, 1460678, 1460679, 1460680, 86885, 74...",1,7.142857,1
-_M-V,1441002,2,"[535657, 1441001, 344456, 394990, 1441003]",1,0.333333,1
._Mathiyalagan,1254327,17,"[1254328, 325462]",1,5.666667,1
075189771_Jeuring,1530795,0,"[1530792, 1530793, 1530794]",0,0.000000,1
2_1,503221,0,"[338655, 503220]",0,0.000000,1


In [42]:
authors_df = authors_df.sort_values(by=['author_id'])
authors_df.head(1000)

,author_id,citations,co_authors,h_index,num_citations,paper_count
Makoto_Satoh,1,102,"[2, 3, 4, 5, 6, 7, 8, 40149, 463789, 463790, 4...",2,24.142857,5
Ryo_Muramatsu,2,0,"[1, 3, 4, 5, 6, 7, 8]",0,0.000000,1
Mizue_Kayama,3,655,"[1, 2, 4, 5, 6, 7, 8, 73800, 30142, 198087, 49...",13,213.900000,32
Kazunori_Itoh,4,364,"[1, 2, 3, 5, 6, 7, 8, 73800, 201615, 334037, 4...",8,93.350000,20
Masami_Hashimoto,5,153,"[1, 2, 3, 4, 6, 7, 8, 201615, 490012, 509990, ...",3,23.100000,10
Makoto_Otani,6,483,"[1, 2, 3, 4, 5, 7, 8, 50209, 324798, 180194, 2...",9,98.450000,39
Michio_Shimizu,7,260,"[1, 2, 3, 4, 5, 6, 8, 201615, 334037, 509990, ...",6,65.833333,10
Masahiko_Sugimoto,8,200,"[1, 2, 3, 4, 5, 6, 7, 509990, 509991, 419420, ...",4,45.833333,7
Gareth_Beale,9,110,"[10, 380963, 380964, 380965, 337260, 178003, 7...",3,31.984127,4
Graeme_Earl,10,565,"[9, 323945, 446287, 446288, 109746, 569693, 56...",8,108.498016,9


In [52]:

authors_df['keywords'] = None
authors_df['affiliation'] = None
'''
def match_names(row):
    global keyword_dict
    print(row['name'])
    if row['name'] in keyword_dict:
        return keyword_dict[row['name']].split(":;:")[1], keyword_dict[row['name']].split(":;:")[0]
    return None, None



authors_df['keywords'], authors_df['affiliation'] = authors_df.apply(lambda x: match_names(x), axis=1)
'''
idx=0
keywords_list = []
affiliations_list = []
for index, row in authors_df.iterrows():
    if index in keyword_dict:
        keywords_list.append(keyword_dict[index].split(":;:")[1])
        affiliations_list.append(keyword_dict[index].split(":;:")[0])
    else:
        keywords_list.append(None)
        affiliations_list.append(None)
        #print(index, row['keywords'], row['affiliation'])
    
        


In [55]:
authors_df['keywords'] =  pd.Series(keywords_list).values
authors_df['affiliation'] =  pd.Series(affiliations_list).values

In [56]:
authors_df.head(100)

,author_id,citations,co_authors,h_index,num_citations,paper_count,keywords,affiliation
Makoto_Satoh,1,102,"[2, 3, 4, 5, 6, 7, 8, 40149, 463789, 463790, 4...",2,24.142857,5,middle school science classroom;science discou...,"Georgia Institute of Technology, Atlanta, GA"
Ryo_Muramatsu,2,0,"[1, 3, 4, 5, 6, 7, 8]",0,0.000000,1,None,None
Mizue_Kayama,3,655,"[1, 2, 4, 5, 6, 7, 8, 73800, 30142, 198087, 49...",13,213.900000,32,collaborative learning;individual learning;sup...,"Senshu University;Shinshu University, Faculty ..."
Kazunori_Itoh,4,364,"[1, 2, 3, 5, 6, 7, 8, 73800, 201615, 334037, 4...",8,93.350000,20,localized sound;movement sense;pattern represe...,"DIINF, University of Santiago of Chile, Chile"
Masami_Hashimoto,5,153,"[1, 2, 3, 4, 6, 7, 8, 201615, 490012, 509990, ...",3,23.100000,10,chorus leader;Tutoring Knowledge;support syste...,"Faculty of Engineering, Shinshu University, Na..."
Makoto_Otani,6,483,"[1, 2, 3, 4, 5, 7, 8, 50209, 324798, 180194, 2...",9,98.450000,39,chorus leader;virtual sound space;Tutoring Kno...,"Toyama Prefectural University, Department of I..."
Michio_Shimizu,7,260,"[1, 2, 3, 4, 5, 6, 8, 201615, 334037, 509990, ...",6,65.833333,10,digital binary pattern;digital figure;segments...,Centro de Investigación Avanzada en Ingeniería...
Masahiko_Sugimoto,8,200,"[1, 2, 3, 4, 5, 6, 7, 509990, 509991, 419420, ...",4,45.833333,7,Othello game;pen input guide;auditory sense;co...,"Graduate School of Science and Technology, Shi..."
Gareth_Beale,9,110,"[10, 380963, 380964, 380965, 337260, 178003, 7...",3,31.984127,4,Roman statue head;accurate visualisation;case ...,"University of Southampton, UK"
Graeme_Earl,10,565,"[9, 323945, 446287, 446288, 109746, 569693, 56...",8,108.498016,9,Minoan cemetery;RTI datasets;RTI practice;anci...,"University of Southampton, Archaeological Comp..."


In [57]:
authors_df.to_csv('./data/authors_with_keywords.csv', sep=',')